### SMS Spam collection
description http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/

Reading the data:

In [25]:
with open("SMSSpamCollection.txt") as f:
    content = f.readlines()

data = [x.strip().split("\t")[1] for x in content]
labels = [x.strip().split("\t")[0] for x in content]

Let's see:

In [26]:
for i in range(0,5):
    print labels[i], data[i]

ham Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
ham Ok lar... Joking wif u oni...
spam Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
ham U dun say so early hor... U c already then say...
ham Nah I don't think he goes to usf, he lives around here though


In [27]:
labels = [1 if l == 'spam' else 0 for l in labels]

Let's extract features from our data using frequency based transformer

In [28]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer

cv = CountVectorizer()
cv.fit(data)
X = cv.transform(data)

Now we evaluate a logistic regression score using f1-metric with 10 cv folds

In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_score

lr = LogisticRegression(random_state=2)
cross_val_score(lr, X, labels, cv=10, scoring='f1').mean()

0.93334852685794145

Good enough! Finally we train our classifier on whole data set and make some predictions:

In [20]:
test_data = ["FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
             "FreeMsg: Txt: claim your reward of 3 hours talk time",
             "Have you visited the last lecture on physics?",
             "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",
             "Only 99$"]

lr.fit(X, labels)
lr.predict(cv.transform(raw_documents=test_data))

array([1, 1, 0, 0, 0])

Good. At least it works. Let's try n-gram approach

In [32]:
for ng in [(2,2), (3,3), (1,3)]:
    print cross_val_score(LogisticRegression(random_state=2), 
                    CountVectorizer(ngram_range=ng).fit_transform(data), 
                    labels, 
                    cv=10, 
                    scoring='f1').mean()

0.822422066419
0.725016155547
0.925138255865


As expected bigram and trigrams don't score well on small sets 

In [36]:
from sklearn.naive_bayes import MultinomialNB

for ng in [(2,2), (3,3), (1,3)]:
    print cross_val_score(MultinomialNB(), 
                    CountVectorizer(ngram_range=ng).fit_transform(data), 
                    labels, 
                    cv=10, 
                    scoring='f1').mean()

0.645455401356
0.378623430876
0.887905460889


We see how badly Naive Bayes works on lack of bigram/trigrams statistics

Finally let's try Tf-idf approach

In [42]:
print cross_val_score(MultinomialNB(), 
                    TfidfVectorizer().fit_transform(data), 
                    labels, 
                    cv=10, 
                    scoring='f1').mean()

0.840253457542
